# Car Price prediction - Part 2

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

Вторая часть соревнования с предсказанием цены автомобиля на основании данных из объявлений о продаже. На этот раз работаем только с готовым датасетом, при этом можно применять модели глубокого обучения, как для анализа табличных данных, так и текста\фотографий.

Ноутбук сделан на основе baseleline, но при этом особое внимание уделено EDA и обработке табличных данных.

In [ ]:
#аугментации изображений
!pip install albumentations -q

In [ ]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import itertools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler


# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers

import albumentations

# plt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
pd.set_option('display.max_columns', 500)

import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords, words
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
!pip freeze > requirements.txt

# Часть 1. EDA.

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [ ]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
train.nunique()

Сразу переименуем столбцы. Названия кириллицей выглядят противоестественно.

In [ ]:
rename = {
    'Владельцы': 'ownersCount',
    'Владение': 'ownershipTime',
    'ПТС': 'titleStatus',
    'Привод': 'driveType',
    'Руль': 'steerSide'
}
train = train.rename(columns = rename)
test = test.rename(columns = rename)

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

Пропуски есть по сути только в одном столбце, ещё в одном в трейне есть 1 пропуск. Дозаполним потом.

## Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [ ]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

## Собственно EDA

Проведем быстрый анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.

Посмотрим, как выглядят распределения числовых признаков:

In [ ]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
    columns = min(3, len(titles_values_dict))
    rows = (len(titles_values_dict) - 1) // columns + 1
    fig = plt.figure(figsize = (columns * 6, rows * 4))
    for i, (title, values) in enumerate(titles_values_dict.items()):
        hist, bins = np.histogram(values, bins = 20)
        ax = fig.add_subplot(rows, columns, i + 1)
        ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
        ax.set_title(title)
    plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

In [ ]:
def numeric_plots(frame, column, nbins = 50):
    with sns.axes_style("white"):
        fig = plt.figure(figsize=(12, 5))
        gs = fig.add_gridspec(1, 2)
        ax1 = fig.add_subplot(gs[0, 0])
        sns.histplot(data=frame, x=frame[column], hue='sample', ax=ax1, bins=nbins)
        ax2 = fig.add_subplot(gs[0, 1])
        sns.boxplot(data=frame[column])
        fig.tight_layout()

In [ ]:
def numeric_hist(frame, column, nbins = 50):
    with sns.axes_style("white"):
        fig = plt.figure(figsize=(5, 5))
        sns.histplot(data=frame, x=frame[column], hue='sample', bins=nbins)
        fig.tight_layout()

Итого:
* CatBoost сможет работать с признаками и в таком виде, но для нейросети нужны нормированные данные.

**Мысли по поводу нормализации датасета** \
Кажется, что совместная нормализация данных трейна и теста некорректна, ведь при подсчёте среднего и стандартного отклонения произойдёт утечка данных из теста в трейн. Корректно было бы определить параметры нормализации на трейне, а потом с их помощью нормализовать тест.
Но тут встаёт другая проблема - если данные трейна и теста были собраны в разное время (как отмечалось на канале в слаке), то такой подход может ухудшить результат, ведь мы будем делать какое-либо машинное обучение на устаревших данных. Поэтому сейчас оставим датасет объединённым, но для целей прода такой подход выглядит некорректным, скорее имеет смысл регулярно переобучать модель на актуальных данных трейна.

In [ ]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

## Обработка числовых признаков

### Пробег

In [ ]:
# Пропусков нет
data.mileage.isna().sum()

In [ ]:
numeric_plots(data, 'mileage')

In [ ]:
#Есть какой-то автомобиль с миллионом км пробега, явный выброс. Что это такое? Вообще посмотрим рекордсменов пробега
data[data.mileage.isin(data.mileage.nlargest(5))]

Очень хочется выкинуть автомобиль с 10000000 км пробега, но он в тесте. А в трейне есть мерседес 2008 года с 999999 км.
Явно автор объявления намеренно ввёл некорректные данные. Надо бы что-то с этими двумя выбросами сделать. Выкидывать не стоит,
ведь один из них в тесте, и к тому же оба принадлежат к довольно редкому классу родстера. Поэтому приравняем их пробег к медиане.
Распределение имеет отрицательное смещение.

**Вместо предложенного в baseline логарифмирования я решил воспользоваться нормализацией через медиану и стандартное отклонение. Этот способ кажется мне лучше, потому что получаемое распределение имеет нулевое среднее и единичное стандартное отклонение.**

In [ ]:
data.loc[data.mileage.isin(data.mileage.nlargest(2)), 'mileage'] = data.mileage.median()

In [ ]:
rscaler = RobustScaler()
data['norm_mileage'] = rscaler.fit_transform(data.mileage.values.reshape(-1,1))
numeric_hist(data, 'norm_mileage')

## Год выпуска модели

Выбросы есть, но они адекватные. Делаем то же самое, что и с моделью

In [ ]:
numeric_plots(data, 'modelDate', nbins = data.modelDate.max() - data.modelDate.min())

In [ ]:
data['norm_modelDate'] = rscaler.fit_transform(data.modelDate.values.reshape(-1,1))
numeric_hist(data, 'norm_modelDate')

## Год выпуска автомобиля

Выбросы есть, но они адекватные.

In [ ]:
numeric_plots(data, 'productionDate', nbins = data.productionDate.max() - data.productionDate.min())

In [ ]:
data['norm_productionDate'] = rscaler.fit_transform(data.productionDate.values.reshape(-1,1))
numeric_hist(data, 'norm_productionDate', nbins = 40)

## Объём двигателя
Посмотрим, какие объёмы вообще бывают.

In [ ]:
data.engineDisplacement.unique()
#есть неопределённый объём двигателя.

In [ ]:
data.query('engineDisplacement == "undefined LTR"')

Понятно, это всё электромобили. Поставим у них объём двигателя 0. Отдельную фичу на электромобиль вводить не будем, это уже заложено в fuelType.
Из остальных значений вытащим объём в литрах как числовое значение.

In [ ]:
data['engineDisplacement'] = data.engineDisplacement.apply(lambda x: float(x.split()[0]) if 'undefined' not in x else 0.0)

In [ ]:
numeric_plots(data, 'engineDisplacement', nbins = 10)

In [ ]:
data['norm_engineDisplacement'] = rscaler.fit_transform(data.engineDisplacement.values.reshape(-1,1))
numeric_hist(data, 'norm_engineDisplacement', nbins = 10)

# Мощность двигателя

In [ ]:
data.enginePower.unique()

In [ ]:
data['enginePower'] = data.enginePower.apply(lambda x: float(x.split()[0]))

In [ ]:
numeric_plots(data, 'enginePower', nbins = 20)

Посмотрим на выбросы свеху.

In [ ]:
data[data.enginePower.isin(data.enginePower.nlargest(5))]

Данные адекватные, оставим.

In [ ]:
data['norm_enginePower'] = rscaler.fit_transform(data.enginePower.values.reshape(-1,1))
numeric_hist(data, 'norm_enginePower', nbins = 20)

## Количество владельцев
Количество владельцев неточное, есть категория 3 и более. По сути тут всего 3 значения. Сделаем это фичей с LabelEncoding. Но и числовое значения нам чуть позже понадобится. Ещё и 1 пропуск есть, заполним модой.

In [ ]:
data.ownersCount.unique()

In [ ]:
data.ownersCount.mode()

In [ ]:
data.loc[data.ownersCount.isna(), 'ownersCount'] = '3 или более'

In [ ]:
data['ownersNumber']=data['ownersCount'].apply(lambda x: int(x.split()[0]))

In [ ]:
numeric_hist(data, 'ownersNumber', 3)

## Время владения
Тут очень много пропусков. Посмотрим в чём дело.

In [ ]:
data[data.ownershipTime.isna()]

Заморочимся и заполним пропуски так:
$$ Время\ владения = 12 \cdot \frac{2021-год\ производства}{количество\ владельцев}$$
Заполненное время владения пересчитаем


In [ ]:
def fill_own_time(row):
    if pd.isna(row['ownershipTime']):
        return 12*(2021-row['productionDate'])/row['ownersNumber']
    elif re.match('.+(лет|год*).*(месяц*)', row['ownershipTime']):
        splitted = row['ownershipTime'].split()
        return 12*int(splitted[0]) + int(splitted[3])
    elif re.match('.+(лет|год*)', row['ownershipTime']):
        splitted = row['ownershipTime'].split()
        return 12*int(splitted[0])
    elif re.match('.+месяц', row['ownershipTime']):
        splitted = row['ownershipTime'].split()
        return int(splitted[0])
    else:
        raise RuntimeError

In [ ]:
data['ownershipTime'] = data.apply(fill_own_time, axis = 1)

In [ ]:
numeric_hist(data,'ownershipTime')

In [ ]:
data['norm_ownershipTime'] = rscaler.fit_transform(data.ownershipTime.values.reshape(-1,1))
numeric_hist(data,'norm_ownershipTime')

## Категориальные признаки.
Количество дверей, владельцев и время владения, хотя они и имеют отношения порядка, всё же отнесём к категориальным признакам. Посмотрим уникальные значения по категориальным фичам.

In [ ]:
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'name','model_info',
                        'numberOfDoors', 'vehicleTransmission', 'ownersCount',
                        'titleStatus', 'driveType', 'steerSide']

In [ ]:
for feature in categorical_features:
    if feature != 'name':
        print(f'***********************{feature}************************')
        print(data[feature].unique())
        print('******************************************************\n\n')

С полем "name" разговор отдельный, слишком много разных имён. Отсюда можно вытащить разные фичи. Пока пойдём наивным путём и посмотрим все "слова" (без цифр), которые встречаются в имени, их не так много на самом деле.

In [ ]:
words = set([x for x in itertools.chain(*list(map(lambda x: x.lower().split(), data.name.unique()))) if x.isalpha()])
words

Нагенерим из этого фичей. Нам надо только выкинуть описание трансмиссии, однобуквенные штуки, повторы и т.п. Вручную зададим список слов-фичей и будем определять, есть ли они в имени, если есть - то ставим фичу = 1 по принципу OneHotEncoding

In [ ]:
keywords = {'activehybrid',
 'amg',
 'blueefficiency',
 'bluetec',
 'cdi',
 'clean',
 'competition',
 'gt',
 'hybrid',
 'long',
 'pack',
 'plus',
 'sdrive',
 'tdi',
 'tfsi',
 'tiptronic',
 'ultra',
 'xdrive',
 'длинный',
 'компактный',
 'экстра'}

In [ ]:
for kwd in keywords:
    data[kwd] = data.name.apply(lambda x: (kwd in x.lower()))

In [ ]:
data['long'] = data['long'] | data['длинный']
data[list(keywords)] = data[list(keywords)].astype(float)
data.rename(columns = {'компактный':'compact',
                       'экстра':'extra'}, inplace=True)
data = data.drop(columns = ['длинный'])

In [ ]:
data.long.unique()

In [ ]:
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info',
                        'numberOfDoors', 'vehicleTransmission', 'ownersCount', 
                        'titleStatus', 'driveType', 'steerSide']
numerical_features = ['mileage', 'modelDate', 'ownersNumber',
                      'productionDate', 'engineDisplacement',
                      'enginePower', 'ownershipTime']
def preproc_data(df_input):
    
    df_output = df_input.copy()
    rscaler = RobustScaler()

    # ################### Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id',], axis = 1, inplace=True)
    
    # ################### Numerical Features ############################################################## 
    df_output.loc[df_output.mileage.isin(df_output.mileage.nlargest(2)), 'mileage'] = df_output.mileage.median()
    df_output.loc[df_output.ownersCount.isna(), 'ownersCount'] = '3 и более'
    
    # Нормализация данных
    df_output['mileage'] = rscaler.fit_transform(df_output.mileage.values.reshape(-1,1))
    df_output['modelDate'] = rscaler.fit_transform(df_output.modelDate.values.reshape(-1,1))
    df_output['productionDate'] = rscaler.fit_transform(df_output.productionDate.values.reshape(-1,1))
    df_output['engineDisplacement'] = df_output.engineDisplacement.apply(lambda x: float(x.split()[0]) if 'undefined' not in x else 0.0)
    df_output['engineDisplacement'] = rscaler.fit_transform(df_output.engineDisplacement.values.reshape(-1,1))
    
    # ################### Feature Engineering ####################################################
    df_output['enginePower'] = df_output.enginePower.apply(lambda x: float(x.split()[0]))
    df_output['enginePower'] = rscaler.fit_transform(df_output.enginePower.values.reshape(-1,1))
    
    df_output['ownersNumber'] = df_output.ownersCount.apply(lambda x: int(x.split()[0]))
    
    def fill_own_time(row):
        if pd.isna(row['ownershipTime']):
            return 12*(2021-row['productionDate'])/row['ownersNumber']
        elif re.match('.+(лет|год*).*(месяц*)', row['ownershipTime']):
            splitted = row['ownershipTime'].split()
            return 12*int(splitted[0]) + int(splitted[3])
        elif re.match('.+(лет|год*)', row['ownershipTime']):
            splitted = row['ownershipTime'].split()
            return 12*int(splitted[0])
        elif re.match('.+месяц', row['ownershipTime']):
            splitted = row['ownershipTime'].split()
            return int(splitted[0])
        else:
            raise RuntimeError
            
    df_output['ownershipTime'] = df_output.apply(fill_own_time, axis = 1)
    df_output['ownershipTime'] = rscaler.fit_transform(df_output.ownershipTime.values.reshape(-1,1))
    
    keywords = ['activehybrid',
                 'amg',
                 'blueefficiency',
                 'bluetec',
                 'cdi',
                 'clean',
                 'competition',
                 'gt',
                 'hybrid',
                 'long',
                 'pack',
                 'plus',
                 'sdrive',
                 'tdi',
                 'tfsi',
                 'tiptronic',
                 'ultra',
                 'xdrive',
                 'длинный',
                 'компактный',
                 'экстра']
    for kwd in keywords:
        df_output[kwd] = df_output.name.apply(lambda x: (kwd in x.lower()))
        
    df_output['long'] = df_output['long'] | df_output['длинный']
    df_output[keywords] = df_output[keywords].astype(float)
    
    df_output.rename(columns = {'компактный':'compact',
                           'экстра':'extra'}, inplace=True)
    df_output = df_output.drop(columns = ['длинный'])
    
    # ################### Categorical Features ############################################################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    # ################### Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    df_output.drop(columns = ['name', 'vehicleConfiguration'], inplace=True)

    return df_output

In [ ]:
#обновим исходный датафрейм
data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

## Корреляции.

Корреляции логичны и ожидаемы - чем старше машина, тем больше у неё было владельцев, больше пробег, старше модель.
Ничего с ними делать не будем.

In [ ]:
sns.heatmap(df_preproc[numerical_features].corr())

## Разделим на трейн и тест

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [ ]:
test_data.info()

# Часть 2. Машинное обучение.
## 2.1. CatBoost.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
cat = CatBoostRegressor(iterations = 7500,
                          depth=5,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          l2_leaf_reg = 2.5,
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          bootstrap_type = 'MVS',
                         )
cat.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=500,
         use_best_model=True,
         )

In [ ]:
test_predict_catboost = cat.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

Вообще-то довольно неплохой результат. Лучше нейросети из бейзлайна.

### Submission

In [ ]:
sub_predict_catboost = cat.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

## 2.2. Tabular NN.

Построим обычную сеть:

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
model.summary()

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

### Fit

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=1000, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [ ]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

## Улучшаем модель.
Методы улучшения:
1) Stack moar layers\
2) Поиграть с регуляризацей в полносвязных слоях\
3) Поиграть с функциями активации (попробовал, не пошло, в ноутбуке нет)\
4) Попробовать слои BactchNorm (помогает)
5) Попробовать поиграть с параметром dropout'a

In [ ]:
better_model = Sequential()
better_model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
better_model.add(L.BatchNormalization())
better_model.add(L.Dropout(0.4))
better_model.add(L.Dense(256, activation="relu"))
better_model.add(L.Dropout(0.4))
better_model.add(L.Dense(128, activation="relu"))
better_model.add(L.Dropout(0.4))
better_model.add(L.Dense(1, activation="linear"))

In [ ]:
better_model.summary()

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
better_model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = better_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=1000, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
test_predict_nn1 = better_model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn1_better = better_model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1_better[:,0]
sample_submission.to_csv('nn1_submission_better.csv', index=False)

## 2.3. NLP + Multiple Inputs

Сначала почистим текст, обрежем окончания, лемматизируем.

In [ ]:
data.description

Почистим и токенизируем текст.

In [ ]:
rubbish = "[A-Za-z!#$%№&'()*+,./:;<=>?@\[\]^_`{|}~—\"\-+●•✅☑️☛\n]"
numbers = '\d+'
df_preproc['description'] = data.description.apply(lambda x: re.sub('\s+', ' ',
                                                               re.sub(numbers, '', 
                                                                    re.sub(rubbish, ' ', x)).lower()))

In [ ]:
df_preproc.description

In [ ]:
lemmer = WordNetLemmatizer()
stemmer = RussianStemmer()
tokenizer = RegexpTokenizer(r'\w+')
def process_words(text):
    tokens = tokenizer.tokenize(text)  
    filtered_words = [word for word in tokens 
                      if len(word) > 2 
                      if word not in stopwords.words('russian')]
    stem_words=[stemmer.stem(word) for word in filtered_words]
    lemma_words=[lemmer.lemmatize(word) for word in stem_words]
    result = " ".join(word for word in lemma_words)
    return result

In [ ]:
df_preproc['description'] = df_preproc.description.apply(process_words)

In [ ]:
df_preproc.description

### Tokenizer

In [ ]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [ ]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df_preproc.description)

In [ ]:
tokenize.word_index

In [ ]:
# split данных
text_train = df_preproc.description.iloc[X_train.index]
text_test = df_preproc.description.iloc[X_test.index]
text_sub = df_preproc.description.iloc[X_sub.index]

In [ ]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [ ]:
# вот так теперь выглядит наш текст
print(text_train.iloc[6])
print(text_train_sequences[6])

### RNN NLP

In [ ]:
# добавил ещё один LSTM - слой к стандартной архитектуре. Он даёт где-то 0.5% на тесте.
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128, return_sequences=True))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.LSTM(64))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(32, activation="relu"))
model_nlp.add(L.Dropout(0.25))

### MLP

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.4))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.4))
model_mlp.add(L.Dense(64, activation="relu"))
model_mlp.add(L.Dropout(0.4))

### Multiple Inputs NN

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
model.summary()

### Fit

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose = 1
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [ ]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission.csv', index=False)

Выглядит так, будто похуже получилось, чем с чистыми табличными данными. Видимо, модель NLP работает не оптимально, требуется дальнейшая настройка.

## 2.4. Добавляем картинки

### Data

In [ ]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

### albumentations

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)


#пример взят из официальной документации: https://albumentations.readthedocs.io/en/latest/examples.html
augmentation = Compose([
    HorizontalFlip(),
    OneOf([
        IAAAdditiveGaussianNoise(),
        GaussNoise(),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.1),
        IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=2),
        IAASharpen(),
        IAAEmboss(),
        RandomBrightnessContrast(),
    ], p=0.3),
    HueSaturationValue(p=0.3),
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
def make_augmentations(images):
    print('применение аугментаций', end = '')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end = '')
    augment_dict = augmentation(image = images[i])
    augmented_image = augment_dict['image']
    augmented_images[i] = augmented_image
    print('')
    return augmented_images

In [ ]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df_preproc.description)

In [ ]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, df_preproc.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, df_preproc.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, df_preproc.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

### Строим сверточную сеть для анализа изображений без "головы"

In [ ]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
#строим нейросеть для анализа табличных данных
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.4))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.4))
model_mlp.add(L.Dense(64, activation="relu"))
model_mlp.add(L.Dropout(0.4))

In [ ]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128, return_sequences=True),
    L.Dropout(0.25),
    L.LSTM(64),
    L.Dropout(0.25),
    L.Dense(32),
    ])

In [ ]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, model_mlp.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, model_mlp.input, nlp_model.input], outputs=head)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

# Blend

In [ ]:
blend_predict = (test_predict_catboost + test_predict_nn1[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
blend_sub_predict = (sub_predict_catboost + sub_predict_nn1_better[:,0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

Простой блендинг не помог, попробуем через метамодель.

In [ ]:
train_cat = cat.predict(X_train).reshape(-1,1)
train_nn = better_model.predict(X_train).reshape(-1,1)
meta_train = np.hstack([train_cat, train_nn])

test_cat = cat.predict(X_test).reshape(-1,1)
test_nn = better_model.predict(X_test).reshape(-1,1)
meta_test = np.hstack([test_cat, test_nn])

In [ ]:
from sklearn.linear_model import LinearRegression
lreg = LinearRegression()
lreg.fit(meta_train, y_train)
meta_test_predict = lreg.predict(meta_test)

In [ ]:
print(f"TEST mape: {(mape(y_test, meta_test_predict))*100:0.2f}%")

Да тоже не особо.

# Выводы.

В отличе от бейзлайна добавление новых признаков в виде текста и фотографий и их обработка нейросетями не даёт положительного эффекта. 
Вероятно, это связано с тем, что эти модели требуют более тонкой настройки и подгонки, чтобы давать результаты, сопоставимые с "простой" нейросетью, которая была настроена.

# Model Bonus: проброс признака

От проброса признака пришлось отказаться, стандартная модель вылетает с ошибкой InvalidArgumentError: 2 root error(s) found на обучении. Разбираться с этим не было времени.